Steps to run inference on [Fashionpedia](https://fashionpedia.github.io/home/index.html)'s attribute_mask_rcnn model and interpret results.

Clone [TensorFlow's TPU repository](https://github.com/tensorflow/tpu/blob/master/README.md), and make sure to have the right tensorflow version installed.

In [ ]:
!git clone https://github.com/tensorflow/tpu/
!pip uninstall tensorflow --y
!pip install tensorflow==2.11

To avoid moduleNotFound errors, add the following to your path:

In [ ]:
import os
# Clean and set PYTHONPATH
os.environ['PYTHONPATH'] = (
    "~/tpu/models:"
    "~/tpu/models/official/detection/dataloader:"
    "~/tpu/models/official/efficientnet:"
    "~/tpu/models/hyperparameters"
)

The file provided to run inference is found in the /tpu/models/official/detection/projects/fashionpedia folder. Make a copy and move it to the detection folder.

In [ ]:
import shutil

source_path = '~/tpu/models/official/detection/projects/fashionpedia/inference.py'
destination_path = '~/tpu/models/official/detection/inferencefp.py'

try:
    shutil.copy(source_path, destination_path)
    print(f"File copied successfully from {source_path} to {destination_path}")
except FileNotFoundError:
    print(f"Source file not found at {source_path}")
except Exception as e:
    print(f"An error occurred: {e}")

To avoid error, line 170 in ~/tpu/models/detection/modeling/base_model.py must be changed.

In [ ]:
file_path = '/content/tpu/models/official/detection/modeling/base_model.py'

# Target line to replace and the replacement text
target_line = 'self._skip_eval_loss = params.eval.skip_eval_loss'
replacement_line = (
    '    try:\n'
    '        self._skip_eval_loss = params.eval.skip_eval_loss\n'
    '    except KeyError as e:\n'
    '        self._skip_eval_loss = False\n'
)


# Read and modify the file
try:
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Replace the target line
    lines = [
        replacement_line if line.strip() == target_line else line
        for line in lines
    ]

    # Write the modified content back to the file
    with open(file_path, 'w') as file:
        file.writelines(lines)

    print("File updated successfully!")
except FileNotFoundError:
    print(f"File not found: {file_path}")
except Exception as e:
    print(f"An error occurred: {e}")


Now we are ready to run the model. First, download checkpoint file for desired backbone at: https://github.com/tensorflow/tpu/tree/master/models/official/detection/projects/fashionpedia. Next, change the path names to your path names in the code below and run.

In [ ]:
!python ~/tpu/models/official/detection/inferencefp.py \
  --model="attribute_mask_rcnn" \
  --image_size=640 \
  # Update with path to checkpoint file
  --checkpoint_path="/path/to/model.ckpt" \
  # Update with name of yaml file
  --config_file="~/tpu/models/official/detection/projects/fashionpedia/configs/yaml/backbone+ID_amrcnn.yaml" \
  # If error with label map, download the file from the repository and change the path to your downloaded file
  --label_map_file="~/tpu/models/official/detection/projects/fashionpedia/dataset/fashionpedia_label_map.csv" \
  # Change jpg to the file type you are using
  --image_file_pattern="/path/to/image/files/*.jpg" \
  # Update with path to store results
  --output_html="/path/to/your/output/folder/results.html" \
  --output_file="/path/to/your/output/folder/results.npy" \
  --max_boxes_to_draw=10 \
  --min_score_threshold=0.05

To interpret category and annotation IDs, download and load the following:

In [ ]:
import numpy as np
import json
import requests

url = "https://s3.amazonaws.com/ifashionist-dataset/annotations/attributes_val2020.json"

response = requests.get(url)

if response.status_code == 200:
    data_attributes = response.json()
else:
    print(f"Failed to download the file, status code: {response.status_code}")
    data_attributes = {}

url2 = "https://s3.amazonaws.com/ifashionist-dataset/annotations/info_test2020.json"

response2 = requests.get(url2)

if response.status_code == 200:
    data = response2.json()
else:
    print(f"Failed to download the file, status code: {response.status_code}")
    data = {}

Next, load your result npy file.

In [ ]:
results = np.load('/path/to/your/output/folder/results.npy', allow_pickle=True)

The following code with print your results.
Example output:
Image:  1
Detected: bag, wallet with score 0.9994686841964722
Bounding box: [870.7718  274.66696 977.52795 316.27097]
Attributes: ID: 204, Name: set-in sleeve, ID: 160, Name: wrist-length, ID: 157, Name: short (length)

In [ ]:
# Extract categories
categories = {category['id']: category for category in data['categories']}

# Extract attributes
attributes = data_attributes.get("attributes", [])

# Iterate through results and match category and attribute IDs to names
count = 1
for result in results:
    print("Image: ", count)
    count += 1
    for cls, box, score, attribute_probs in zip(result["classes"], result["boxes"], result["scores"], result["attributes"]):
        # Get the category name
        category_name = categories.get(cls, {}).get('name', 'Unknown')

        # Get the attributes sorted by probability
        attribute_ids = np.argsort(attribute_probs)[::-1]  # Sort attributes by probability
        attribute_info = []

        for attr_id in attribute_ids:
            # Get the attribute name using the attribute ID from data_attributes
            if 0 <= attr_id < len(attributes):  # Ensure the attribute ID is valid
                name = attributes[attr_id].get('name', 'Unknown')
            else:
                name = 'Unknown'

            # If name is 'Unknown', still show the ID
            if name == 'Unknown':
                attribute_info.append(f"ID: {attr_id}, Name: Unknown")
            else:
                attribute_info.append(f"ID: {attr_id}, Name: {name}")

        # Print the results with category and attribute names
        print(f"Detected: {category_name} with score {score}")
        print(f"Bounding box: {box}")
        print(f"Attributes: {', '.join(attribute_info) if attribute_info else 'No attributes detected'}")
